## Molecular Dynamics-Inspired Proxy Workflow

This workflow uses the framework from the NSF-funded Analytics4MD project (https://analytics4md.org/) to create a proxy for a common data movement pattern in Molecular Dynamics (MD) workflows. The figure below shows how the A4MD framework operates at a high level:

![a4md](a4md_wflow.png)

### Set configuration parameters

Set the directories to be used in this demo

In [ ]:
producer_staging_directory = "/tmp/md_prod"
consumer_staging_directory = "/tmp/md_cons"
shared_staging_directory = "/tmp/md_shared"

Set a namespace in the Flux KVS that DYAD will use

In [ ]:
kvs_namespace = "dyad_md_example"

Set the application parameters. For this simple point-to-point workflow, there are two:
1. `num_frames`: the number of frames (i.e., snapshots of the molecular system) to be produced/consumed
2. `num_atoms`: the number of atoms in the molecular system

In [ ]:
num_frames = 10
num_atoms = 200

Set the paths to the Python modules implementing the producer and consumer applications

In [ ]:
py_generation_module = "/usr/libexec/load.py"
py_analysis_module = "/usr/libexec/compute.py"

Set the entrypoint functions for the producer and consumer applications

In [ ]:
py_generation_func = "extract_frame"
py_analysis_func = "analyze"

Set the paths to the libraries implementing DYAD's service and DYAD's I/O interception

In [ ]:
dyad_module = "/usr/lib/x86_64-linux-gnu/dyad.so"
dyad_wrapper_library = "/usr/lib/x86_64-linux-gnu/dyad_wrapper.so"

Create the staging directories specified above

In [ ]:
!rm -rf {producer_staging_directory}
!mkdir -p {producer_staging_directory}
!chmod 755 {producer_staging_directory}
!rm -rf {consumer_staging_directory}
!mkdir -p {consumer_staging_directory}
!chmod 755 {consumer_staging_directory}
!rm -rf {shared_staging_directory}
!mkdir -p {shared_staging_directory}
!chmod 755 {shared_staging_directory}

### MD Workflow using Local Storage

This run of the workflow assumes that the generated data will be saved in local storage. In this case, DYAD will provide two services to the workflow:
1. DYAD will block the consumer until data is made available
2. DYAD will transfer the data from the producer's local storage to the consumer's local storage when the consumer tries to access the data

Since this example is running in Docker, the producer and consumer are running on the same "node". To emulate the use of node-local storage in HPC, we specify separate "managed" directories for the producer and consumer. When running on an actual HPC system, the "managed" directories would be the same for both producer and consumer, and they would point to some node-local storage resource (e.g., `/tmp`, `/p/ssd`). 

Start the Flux KVS namespace to be used by DYAD in the workflow

In [ ]:
!flux kvs namespace create {kvs_namespace}
!flux kvs namespace list

Start the DYAD service

In [ ]:
!flux exec -r all flux module load {dyad_module} {producer_staging_directory}

In [ ]:
!flux exec -r 0 flux module list

Generate the commands to launch producer and consumer

In [ ]:
producer_cmd = "LD_PRELOAD={wrapper_lib} DYAD_KVS_NAMESPACE={kvs_namespace} \
DYAD_PATH_PRODUCER={producer_dir} DYAD_DTL_MODE=UCX flux run -N 1 -n 1 \
/usr/bin/balanced_producer {gen_module_path} {gen_func} {n_frames} {n_atoms} \
0 -d {producer_dir}".format(
    wrapper_lib=dyad_wrapper_library,
    kvs_namespace=kvs_namespace,
    producer_dir=producer_staging_directory,
    gen_module_path=py_generation_module,
    gen_func=py_generation_func,
    n_frames=num_frames,
    n_atoms=num_atoms,
)
print(producer_cmd)

In [ ]:
consumer_cmd = "LD_PRELOAD={wrapper_lib} DYAD_KVS_NAMESPACE={kvs_namespace} \
DYAD_PATH_CONSUMER={consumer_dir} DYAD_DTL_MODE=UCX flux run -N 1 -n 1 \
/usr/bin/balanced_consumer {analysis_mod_path} {analysis_func} {n_frames} \
-d {consumer_dir}".format(
    wrapper_lib=dyad_wrapper_library,
    kvs_namespace=kvs_namespace,
    consumer_dir=consumer_staging_directory,
    analysis_mod_path=py_analysis_module,
    analysis_func=py_analysis_func,
    n_frames=num_frames
)
print(consumer_cmd)

Launch the workflow tasks

Since this demo is running in Jupyter, launching the consumer first will block other cells in the notebook from running. Due to this, the producer must be launched in a terminal.

In [ ]:
!{consumer_cmd}

In [ ]:
# Run producer_cmd in terminal

Confirm that the data transfer happened

In [ ]:
!ls -lah {producer_staging_directory}/group0

In [ ]:
!ls -lah {consumer_staging_directory}/group0

Cleanup the Flux KVS and shutdown the DYAD service

In [ ]:
!flux exec -r all flux module unload dyad

In [ ]:
!echo "Modules Post-Cleanup"
!echo "===================="
!flux module list

In [ ]:
!flux kvs namespace remove {kvs_namespace}

In [ ]:
!echo "KVS Namespaces Post-Cleanup"
!echo "==========================="
!flux kvs namespace list

### MD Workflow using Shared Storage

This run of the workflow assumes that the generated data will be saved in shared storage (e.g., `/p/lustre1` on LC). In this case, DYAD will only provide consumer blocking. It will not transfer data.

Create the KVS namespace and start the DYAD service

In [ ]:
!flux kvs namespace create {kvs_namespace}
!flux exec -r all flux module load {dyad_module} {shared_staging_directory}

Generate commands to launch the producer and consumer

In [ ]:
producer_cmd = "LD_PRELOAD={wrapper_lib} DYAD_KVS_NAMESPACE={kvs_namespace} \
DYAD_PATH_PRODUCER={producer_dir} DYAD_SHARED_STORAGE=1 DYAD_DTL_MODE=UCX \
flux run -N 1 -n 1 \
/usr/bin/balanced_producer {gen_module_path} {gen_func} {n_frames} {n_atoms} \
0 -d {producer_dir}".format(
    wrapper_lib=dyad_wrapper_library,
    kvs_namespace=kvs_namespace,
    producer_dir=shared_staging_directory,
    gen_module_path=py_generation_module,
    gen_func=py_generation_func,
    n_frames=num_frames,
    n_atoms=num_atoms,
)
print(producer_cmd)

In [ ]:
consumer_cmd = "LD_PRELOAD={wrapper_lib} DYAD_KVS_NAMESPACE={kvs_namespace} \
DYAD_PATH_CONSUMER={consumer_dir} DYAD_SHARED_STORAGE=1 DYAD_DTL_MODE=UCX \
flux run -N 1 -n 1 \
/usr/bin/balanced_consumer {analysis_mod_path} {analysis_func} {n_frames} \
-d {consumer_dir}".format(
    wrapper_lib=dyad_wrapper_library,
    kvs_namespace=kvs_namespace,
    consumer_dir=shared_staging_directory,
    analysis_mod_path=py_analysis_module,
    analysis_func=py_analysis_func,
    n_frames=num_frames
)
print(consumer_cmd)

Launch the workflow tasks

In [ ]:
!{consumer_cmd}

In [ ]:
# Run producer_cmd in terminal

Check that the data is in the shared storage directory

In [ ]:
!ls -lah {shared_staging_directory}/group0

Cleanup the Flux KVS and shutdown the DYAD service

In [ ]:
!flux exec -r all flux module unload dyad

In [ ]:
!echo "Modules Post-Cleanup"
!echo "===================="
!flux module list

In [ ]:
!flux kvs namespace remove {kvs_namespace}

In [ ]:
!echo "KVS Namespaces Post-Cleanup"
!echo "==========================="
!flux kvs namespace list